In [4]:
import os 
import pandas as pd
import numpy as np
from scipy.io import savemat
from utils.utils import *
from utils.flips import get_prefered_xy

### Matlab export notebook
This notebook contains cells that extract data from the models and export them in both matlab and csv format; which are then used for statistical anaylsis 

Current format is a bit messy, so things might be broken down the line 

If I wasnt lazy I would send all these directly to the right folder, but I am lazy 

In [10]:
"""
Threshold is used to determine which data is from the lh, and which is from the rh, since we want to check if hemis is a varible with a satstically significant impact 
"""
import nibabel as nib
threshold = []
for sub in subj_list:
    maskdata_lh_path = os.path.join(mask_dir, sub , f'lh.{sub}.testrois.mgz')
    maskdata_lh = nib.load(maskdata_lh_path).get_fdata().squeeze()
    lh_indices = np.where((maskdata_lh >= 1) & (maskdata_lh <= 15))[0] 
    threshold.append(lh_indices.shape[0])

threshold
    

[17901, 19290, 17885, 13606, 14050, 19115, 13976, 15588]

In [12]:
cos_sin = get_prefered_xy(subj_list, sessions, fetch_conds=False) 
models_subs = {}
models = ['best_roi']
columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang", "roi", "best_roi"]
model_study = 'oself'
mode = "train"
rotated = True

for i, sub in enumerate(subj_list):
    models_subs[sub] = {}
    noise_ceilling_file = os.path.join(noise_dir, f'{sub}_noise_ceilling_all_vox.npy')
    noise_ceilling = np.load(noise_ceilling_file, allow_pickle=True)
    for m in models:
        if rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}.npy')
        if not rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_notrotated.npy')
        models_subs[sub][m] = pd.DataFrame(np.load(m_file, allow_pickle=True), columns=columns)
        models_subs[sub][m][columns[:-2]] = models_subs[sub][m][columns[:-2]].astype(np.float32)
        models_subs[sub][m]['noise_ceilling'] = noise_ceilling
        models_subs[sub][m]['voxel_performance'] = models_subs[sub][m]["test_var_explained"] / models_subs[sub][m]['noise_ceilling']
        # this should be okay 

models_subs_bestroi = models_subs
#savemat('models_subs_notrotated.mat', models_subs_notrotated)
savemat('models_subs_bestroi.mat', models_subs_bestroi)


(907,)


In [16]:
rotated = True
models = ['best_roi']
columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang",  "roi", "best_roi"]
for i, sub in enumerate(subj_list):
    models_subs[sub] = {}
    noise_ceilling_file = os.path.join(noise_dir, f'{sub}_noise_ceilling_all_vox.npy')
    noise_ceilling = np.load(noise_ceilling_file, allow_pickle=True)
    for m in models:
        if rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}.npy')
        if not rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_notrotated.npy')

        model = np.load(m_file, allow_pickle=True)
        models_subs[sub][m] = {}
        for hemi in hemis:
            
            if hemi == "lh":
                models_subs[sub][m][hemi] = pd.DataFrame(model[:threshold[i]], columns=columns)
            if hemi == "rh":
                models_subs[sub][m][hemi] = pd.DataFrame(model[threshold[i]:], columns=columns)
            models_subs[sub][m][hemi][columns[:-2]] = models_subs[sub][m][hemi][columns[:-2]].astype(np.float32)
            if hemi == 'lh': 
                models_subs[sub][m][hemi]['noise_ceilling'] = noise_ceilling[:threshold[i]]
            if hemi == 'rh': 
                models_subs[sub][m][hemi]['noise_ceilling'] = noise_ceilling[threshold[i]:]
# no need to get noise ceilling here

"""-------------VARIANCE----------"""
variance_lh = np.zeros((len(subj_list), len(rois.keys())))
m = models[0]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_lh[i, j] = models_subs_bestroi[sub][m]["lh"].groupby('roi')['test_var_explained'].mean()[roi]


var_df_lh = pd.DataFrame(variance_lh, index=subj_list, columns=rois.keys())
var_df_lh.to_csv('test_variance_bestroi_lh.csv', index=True)

variance_rh = np.zeros((len(subj_list), len(rois.keys())))
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_rh[i, j] = models_subs_bestroi[sub][m]["rh"].groupby('roi')['test_var_explained'].mean()[roi]


var_df_rh = pd.DataFrame(variance_rh, index=subj_list, columns=rois.keys())
var_df_rh.to_csv('test_variance_bestroi_rh.csv', index=True)


"""--------SIGMA------"""
sigma_lh = np.zeros((len(subj_list), len(rois.keys())))
m = models[0]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        sigma_lh[i, j] = models_subs_bestroi[sub][m]["lh"].groupby('roi')['sigma'].mean()[roi]


sigma_df_lh = pd.DataFrame(sigma_lh, index=subj_list, columns=rois.keys())
sigma_df_lh.to_csv('sigma_bestroi_lh.csv', index=True)

sigma_rh = np.zeros((len(subj_list), len(rois.keys())))
m = models[0]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        sigma_rh[i, j] = models_subs_bestroi[sub][m]["rh"].groupby('roi')['sigma'].mean()[roi]


sigma_df_rh = pd.DataFrame(sigma_rh, index=subj_list, columns=rois.keys())
sigma_df_rh.to_csv('sigma_bestroi_rh.csv', index=True)

'--------NOISE CEILLING-------'

noiseceiling_lh = np.zeros((len(subj_list), len(rois.keys())))
m = models[0]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        noiseceiling_lh[i, j] = models_subs_bestroi[sub][m]["lh"].groupby('roi')['noise_ceilling'].mean()[roi]


nc_df_lh = pd.DataFrame(noiseceiling_lh, index=subj_list, columns=rois.keys())
nc_df_lh.to_csv('noiseceiling_bestroi_lh.csv', index=True)

noiseceiling_rh = np.zeros((len(subj_list), len(rois.keys())))
m = models[0]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        noiseceiling_rh[i, j] = models_subs_bestroi[sub][m]["rh"].groupby('roi')['noise_ceilling'].mean()[roi]


nc_df_rh = pd.DataFrame(noiseceiling_rh, index=subj_list, columns=rois.keys())
nc_df_rh.to_csv('noiseceiling_bestroi_rh.csv', index=True)

In [5]:
rotated = True

for i, sub in enumerate(subj_list):
    models_subs[sub] = {}
    for m in models:
        if rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_rotated.npy')
        if not rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_notrotated.npy')
        models_subs[sub][m] = pd.DataFrame(np.load(m_file, allow_pickle=True), columns=columns)
        models_subs[sub][m][columns[:-2]] = models_subs[sub][m][columns[:-2]].astype(np.float32)


models_subs_rotated = models_subs
savemat('models_subs_rotated.mat', models_subs_rotated)
models_subs_rotated

{'subj01': {'oself':              x0        y0     sigma      slope  intercept  test_var_explained  \
  0      0.371848 -0.624069  6.235634  30.357092 -30.137138            0.000676   
  1      0.418255 -0.594880  6.208899  25.832947 -25.632463           -0.000803   
  2      0.502787 -0.546511  4.808846  12.609944 -12.428826           -0.002064   
  3      0.349385 -0.618350  5.959802  28.299004 -28.080090           -0.001289   
  4      0.331797 -0.591528  6.096788  30.145597 -29.928112           -0.002998   
  ...         ...       ...       ...        ...        ...                 ...   
  33792  0.141309 -1.050000  5.095603  48.103619 -47.682968            0.010695   
  33793 -0.109746 -1.050000  3.967579  16.174715 -15.943175            0.005295   
  33794 -0.283447  0.284247  4.676534  14.986609 -14.808397            0.008918   
  33795 -0.195607 -0.807038  4.128424  15.912541 -15.684640            0.006280   
  33796 -0.314990 -0.148864  6.484445  29.390841 -29.198298         

In [30]:
rotated = True
models = ['wself' ,'oself']
for i, sub in enumerate(subj_list):
    models_subs[sub] = {}
    for m in models:
        if rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_rotated.npy')
        if not rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_notrotated.npy')

        model = np.load(m_file, allow_pickle=True)
        models_subs[sub][m] = {}
        for hemi in hemis:
            
            if hemi == "lh":
                models_subs[sub][m][hemi] = pd.DataFrame(model[:threshold[i]], columns=columns)
            if hemi == "rh":
                models_subs[sub][m][hemi] = pd.DataFrame(model[threshold[i]:], columns=columns)
            models_subs[sub][m][hemi][columns[:-2]] = models_subs[sub][m][hemi][columns[:-2]].astype(np.float32)


variance_lh = np.zeros((len(subj_list), len(rois.keys())))
m = models[1]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_lh[i, j] = models_subs[sub][m]["lh"].groupby('roi')['var_explained'].mean()[roi]


var_df_lh = pd.DataFrame(variance_lh, index=subj_list, columns=rois.keys())
var_df_lh.to_csv('variance_rotated_lh.csv', index=True)

variance_rh = np.zeros((len(subj_list), len(rois.keys())))
m = models[1]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_rh[i, j] = models_subs[sub][m]["rh"].groupby('roi')['var_explained'].mean()[roi]


var_df_rh = pd.DataFrame(variance_rh, index=subj_list, columns=rois.keys())
var_df_rh.to_csv('variance_rotated_rh.csv', index=True)


variance_lh = np.zeros((len(subj_list), len(rois.keys())))
m = models[1]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_lh[i, j] = models_subs[sub][m]["lh"].groupby('roi')['test_var_explained'].mean()[roi]


var_df_lh = pd.DataFrame(variance_lh, index=subj_list, columns=rois.keys())
var_df_lh.to_csv('test_variance_rotated_lh.csv', index=True)

variance_rh = np.zeros((len(subj_list), len(rois.keys())))
m = models[1]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_rh[i, j] = models_subs[sub][m]["rh"].groupby('roi')['test_var_explained'].mean()[roi]


var_df_rh = pd.DataFrame(variance_rh, index=subj_list, columns=rois.keys())
var_df_rh.to_csv('test_variance_rotated_rh.csv', index=True)
var_df_rh

####
sigma_lh = np.zeros((len(subj_list), len(rois.keys())))
m = models[1]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        sigma_lh[i, j] = models_subs[sub][m]["lh"].groupby('roi')['sigma'].mean()[roi]


sigma_df_lh = pd.DataFrame(sigma_lh, index=subj_list, columns=rois.keys())
sigma_df_lh.to_csv('sigma_rotated_lh.csv', index=False)

sigma_rh = np.zeros((len(subj_list), len(rois.keys())))
m = models[1]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        sigma_rh[i, j] = models_subs[sub][m]["rh"].groupby('roi')['sigma'].mean()[roi]


sigma_df_rh = pd.DataFrame(sigma_rh, index=subj_list, columns=rois.keys())
sigma_df_rh.to_csv('sigma_rotated_rh.csv', index=False)

In [26]:
models_subs = {}
models = ['oself']
columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang", "roi", "fit_with"]
model_study = 'oself'
mode = "train"
hemis = ["lh", "rh"]
rotated = False

for i, sub in enumerate(subj_list):
    models_subs[sub] = {}
    for m in models:
        if rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_rotated.npy')
        if not rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_notrotated.npy')

        model = np.load(m_file, allow_pickle=True)
        models_subs[sub][m] = {}
        for hemi in hemis:
            
            if hemi == "lh":
                models_subs[sub][m][hemi] = pd.DataFrame(model[:threshold[i]], columns=columns)
            if hemi == "rh":
                models_subs[sub][m][hemi] = pd.DataFrame(model[threshold[i]:], columns=columns)
            models_subs[sub][m][hemi][columns[:-2]] = models_subs[sub][m][hemi][columns[:-2]].astype(np.float32)



variance_lh = np.zeros((len(subj_list), len(rois.keys())))
m = models[0]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_lh[i, j] = models_subs[sub][m]["lh"].groupby('roi')['var_explained'].mean()[roi]


var_df_lh = pd.DataFrame(variance_lh, index=subj_list, columns=rois.keys())
var_df_lh.to_csv('variance_notrotated_lh.csv', index=True)

variance_rh = np.zeros((len(subj_list), len(rois.keys())))
m = models[0]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_rh[i, j] = models_subs[sub][m]["rh"].groupby('roi')['var_explained'].mean()[roi]


var_df_rh = pd.DataFrame(variance_rh, index=subj_list, columns=rois.keys())
var_df_rh.to_csv('variance_notrotated_rh.csv', index=True)


variance_lh = np.zeros((len(subj_list), len(rois.keys())))
m = models[0]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_lh[i, j] = models_subs[sub][m]["lh"].groupby('roi')['test_var_explained'].mean()[roi]


var_df_lh = pd.DataFrame(variance_lh, index=subj_list, columns=rois.keys())
var_df_lh.to_csv('test_variance_notrotated_lh.csv', index=True)

variance_rh = np.zeros((len(subj_list), len(rois.keys())))
m = models[0]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_rh[i, j] = models_subs[sub][m]["rh"].groupby('roi')['test_var_explained'].mean()[roi]


var_df_rh = pd.DataFrame(variance_rh, index=subj_list, columns=rois.keys())
var_df_rh.to_csv('test_variance_notrotated_rh.csv', index=True)
var_df_rh

##################

sigma_lh = np.zeros((len(subj_list), len(rois.keys())))
m = models[0]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        sigma_lh[i, j] = models_subs[sub][m]["lh"].groupby('roi')['sigma'].mean()[roi]


sigma_df_lh = pd.DataFrame(sigma_lh, index=subj_list, columns=rois.keys())
sigma_df_lh.to_csv('sigma_notrotated_lh.csv', index=False)

sigma_rh = np.zeros((len(subj_list), len(rois.keys())))
m = models[0]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        sigma_rh[i, j] = models_subs[sub][m]["rh"].groupby('roi')['sigma'].mean()[roi]


sigma_df_rh = pd.DataFrame(sigma_rh, index=subj_list, columns=rois.keys())
sigma_df_rh.to_csv('sigma_notrotated_rh.csv', index=False)

